In [1]:
# Import necessary functions
import os
import aryn_sdk
from aryn_sdk.partition import partition_file, draw_with_boxes, convert_image_element
import json
import pdf2image
#from openai import OpenAI
from pydantic import BaseModel
#from openai.lib._parsing import type_to_response_format_param as pydantic_to_response_format
#from google.colab import userdata
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Set your secrets in the colab notebook. Navigate to the left pane
# and choose the key option to set your keys. Make sure to enable Notebook access

# Visit https://www.aryn.ai/get-started to get a key.
aryn_api_key = os.getenv('aryn_API_KEY')
# Visit https://platform.openai.com/api-keys to get a key.
openai_api_key = os.getenv('anthropic_API_KEY')

In [3]:
file_name = 'syllabus122.pdf'
file = open(file_name, 'rb')

#show the pdf
#pdf2image.convert_from_path(file_name)[0]

In [4]:
## Make a call to the partitioning service and set extract_images to true.
partitioned_file = partition_file(file, aryn_api_key, extract_images=True, extract_table_structure=True, use_ocr=True)

# show the pdf with bounding boxes
#draw_with_boxes(file_name, partitioned_file)[0]

ArynPartitioner: T+   0.00: Server version aryn-partitioner-0.20241226.161207 Model version 1.4
ArynPartitioner: T+   0.00: Received request with aryn_call_id=a5abebb6-38c3-4a0d-ba8f-41b4ee91b357
ArynPartitioner: T+   0.00: Waiting for scheduling
ArynPartitioner: T+   0.00: Preprocessing document
ArynPartitioner: T+   0.00: Done preprocessing document
ArynPartitioner: T+   1.01: Completed work on page 1
ArynPartitioner: T+   1.23: Completed work on page 4
ArynPartitioner: T+   1.66: Completed work on page 5
ArynPartitioner: T+   1.87: Completed work on page 3
ArynPartitioner: T+   2.08: Completed work on page 6
ArynPartitioner: T+   2.34: Completed work on page 2


In [5]:
## Walk over the image elements in the returned JSON and show the first one
data = partitioned_file['elements']
print(data)

[{'type': 'Section-header', 'bbox': [0.11799425012925092, 0.09089284723455256, 0.6248676614200368, 0.12518794666637073], 'properties': {'score': 0.4963124692440033, 'page_number': 1}, 'text_representation': 'CS 122A/EECS 116 Introduction to Data Management Winter 2025'}, {'type': 'Text', 'bbox': [0.11958067052504595, 0.12616450916637073, 0.5697048770680148, 0.14329833984375], 'properties': {'score': 0.360942542552948, 'page_number': 1}, 'text_representation': 'nstructor: Prof. Faisal Nawab <nawabfauci.edu.'}, {'type': 'Text', 'bbox': [0.17535136503331802, 0.14281638405539773, 0.6240349264705882, 0.16098384510387073], 'properties': {'score': 0.3371292054653168, 'page_number': 1}, 'text_representation': 'Office hours: Tuesdays 12:30 - 1:30PM, DBH 2088'}, {'type': 'List-item', 'bbox': [0.11930120131548713, 0.16028894597833807, 0.4921765495749081, 0.2651818570223722], 'properties': {'score': 0.5536366701126099, 'page_number': 1}, 'text_representation': 'TAs: Farzad Habibi <habibif@uci.edu>

In [6]:
# Pydantic models used to form response_format parameter to anthropic
class description(BaseModel):
    image_title: str
    image_description: str

class diagram_response(BaseModel):
    steps: list[description]

In [7]:
diagram_response_schema = diagram_response.model_json_schema()

tools = [
    {
        "name": "build_description_result",
        "description": "build the description object",
        "input_schema": diagram_response_schema
    }
]

In [8]:
def format_content(structured_content):
    formatted_text = ""
    for item in structured_content:
        if item.get("type") in ["Section-header", "Text", "List-item"]:
            formatted_text += f"{item['text_representation']}\n\n"
        elif item.get("type") == "table":
            formatted_text += "Table:\n"
            for cell in item['table']['cells']:
                formatted_text += f"{cell['content']} | "
            formatted_text += "\n\n"
    return formatted_text

formatted_content = format_content(data)
print(data)

[{'type': 'Section-header', 'bbox': [0.11799425012925092, 0.09089284723455256, 0.6248676614200368, 0.12518794666637073], 'properties': {'score': 0.4963124692440033, 'page_number': 1}, 'text_representation': 'CS 122A/EECS 116 Introduction to Data Management Winter 2025'}, {'type': 'Text', 'bbox': [0.11958067052504595, 0.12616450916637073, 0.5697048770680148, 0.14329833984375], 'properties': {'score': 0.360942542552948, 'page_number': 1}, 'text_representation': 'nstructor: Prof. Faisal Nawab <nawabfauci.edu.'}, {'type': 'Text', 'bbox': [0.17535136503331802, 0.14281638405539773, 0.6240349264705882, 0.16098384510387073], 'properties': {'score': 0.3371292054653168, 'page_number': 1}, 'text_representation': 'Office hours: Tuesdays 12:30 - 1:30PM, DBH 2088'}, {'type': 'List-item', 'bbox': [0.11930120131548713, 0.16028894597833807, 0.4921765495749081, 0.2651818570223722], 'properties': {'score': 0.5536366701126099, 'page_number': 1}, 'text_representation': 'TAs: Farzad Habibi <habibif@uci.edu>

In [18]:
import anthropic

client = anthropic.Anthropic(api_key=openai_api_key)
message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=8192,
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "The following is a PDF syllabus document. Please extract specific key dates and times of relevant events like lectures, midterms, and project deadlines from the document. "
    "Use the provided template to create a calendar event for each relevant date.\n\n"
    "If an event does not have specific information for any field, set that field to `null`.\n\n"
    "Template:\n"
    "event = {\n"
    "  'summary': 'Event summary (e.g., Midterm Exam)',\n"
    "  'location': 'Event location (if provided)',\n"
    "  'description': 'Brief description of the event',\n"
    "  'start': {\n"
    "    'dateTime': 'Start date and time in ISO 8601 format (e.g., 2025-05-28T09:00:00-07:00)',\n"
    "    'timeZone': 'Time zone (if provided)',\n"
    "  },\n"
    "  'end': {\n"
    "    'dateTime': 'End date and time in ISO 8601 format (if available)',\n"
    "    'timeZone': 'Time zone (if provided)',\n"
    "  },\n"
    "  'recurrence': [\n"
    "    'Recurrence rule in RRULE format (if applicable)',\n"
    "  ],\n"
    "  'attendees': [\n"
    "    {'email': 'Attendee email (if available)'},\n"
    "  ],\n"
    "  'reminders': {\n"
    "    'useDefault': False,\n"
    "    'overrides': [\n"
    "      {'method': 'email', 'minutes': 24 * 60},\n"
    "      {'method': 'popup', 'minutes': 10},\n"
    "    ],\n"
    "  },\n"
    "}\n\n"
    "Document:\n"
    f"{formatted_content}\n\n"
    "Instructions:\n"
    "1. Extract key events such as lectures, midterms, finals, and project deadlines.\n"
    "2. Fill out the `summary`, `start`, `end`, `location`, and `description` fields based on the document content.\n"
    "3. If the time zone is not mentioned, set it to `null`.\n"
    "4. If recurrence information (like repeating lectures) is provided, use the `recurrence` field with an RRULE.\n"
    "5. If no attendees are listed, set the `attendees` field to an empty list.\n"
    "6. Set any missing fields to `null` if the information is unavailable.\n"
    "7. Use ISO 8601 format for date and time fields."
    "Give no further text only the filled out events."
                }
            ],
        }
    ],
        tools=tools,
        tool_choice={"type": "tool", "name": "build_description_result"}

)
print(message)


Message(id='msg_013jJf8MPzT8q3FgBqy3VVYC', content=[ToolUseBlock(id='toolu_01FDKPqiezvpvWFMFdgZmUjS', input={'steps': [{'image_title': 'Lectures', 'image_description': "{\n      'summary': 'CS 122A/EECS 116 Lecture',\n      'location': 'HSLH 100A',\n      'description': 'Introduction to Data Management lecture',\n      'start': {\n        'dateTime': '2025-01-07T11:00:00-08:00',\n        'timeZone': 'America/Los_Angeles'\n      },\n      'end': {\n        'dateTime': '2025-01-07T12:20:00-08:00',\n        'timeZone': 'America/Los_Angeles'\n      },\n      'recurrence': [\n        'RRULE:FREQ=WEEKLY;BYDAY=TU,TH;UNTIL=20250321T235959Z'\n      ],\n      'attendees': [],\n      'reminders': {\n        'useDefault': false,\n        'overrides': [\n          {'method': 'email', 'minutes': 1440},\n          {'method': 'popup', 'minutes': 10}\n        ]\n      }\n    }"}, {'image_title': 'Instructor Office Hours', 'image_description': "{\n      'summary': 'Prof. Nawab Office Hours',\n      'loc

In [19]:
out = json.dumps(message.content[0].input['steps'], indent=4)
print(out)
def printout(jsonin):
    json_data = json.loads(jsonin)

    for dic in json_data:
        print(f"{dic['image_title']}: \n{dic['image_description']}\n")
printout(out)

Lectures: 
{
      'summary': 'CS 122A/EECS 116 Lecture',
      'location': 'HSLH 100A',
      'description': 'Introduction to Data Management lecture',
      'start': {
        'dateTime': '2025-01-07T11:00:00-08:00',
        'timeZone': 'America/Los_Angeles'
      },
      'end': {
        'dateTime': '2025-01-07T12:20:00-08:00',
        'timeZone': 'America/Los_Angeles'
      },
      'recurrence': [
        'RRULE:FREQ=WEEKLY;BYDAY=TU,TH;UNTIL=20250321T235959Z'
      ],
      'attendees': [],
      'reminders': {
        'useDefault': false,
        'overrides': [
          {'method': 'email', 'minutes': 1440},
          {'method': 'popup', 'minutes': 10}
        ]
      }
    }

Instructor Office Hours: 
{
      'summary': 'Prof. Nawab Office Hours',
      'location': 'DBH 2088',
      'description': 'Professor office hours',
      'start': {
        'dateTime': '2025-01-07T12:30:00-08:00',
        'timeZone': 'America/Los_Angeles'
      },
      'end': {
        'dateTime': '2025

In [11]:

message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=8192,
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "The following is a PDF syllabus document converted using aryn docparse. Please give a summary of all of the important information"
    f"{formatted_content}\n\n"
                }
            ],
        }
    ],
        tools=tools,
        tool_choice={"type": "tool", "name": "build_description_result"}

)
print(message)


Message(id='msg_01SCAQjqAqtkAwxPyXZkd3fe', content=[ToolUseBlock(id='toolu_01Q729TwXWfD5kA2itZ24JWp', input={'steps': [{'image_title': 'Course Information Summary', 'image_description': 'CS 122A/EECS 116 Introduction to Data Management - Winter 2025\nInstructor: Prof. Faisal Nawab\nLectures: TuTh 11AM-12:20PM at HSLH 100A\nOffice Hours: Tuesdays 12:30-1:30PM, DBH 2088'}, {'image_title': 'Course Components and Grading', 'image_description': '- Homework Assignments (40%)\n- Two Midterm Exams (20% each)\n- Project (15%)\n- Participation (5%)\nLate Policy: Up to 2 homework assignments can be submitted up to 24 hours late using late passes'}, {'image_title': 'Learning Objectives', 'image_description': '1. Design E-R diagrams\n2. Build DB relations using relational data model\n3. Apply database design principles\n4. Use relational algebra\n5. Create and use SQL\n6. Understand indexing and physical database design'}, {'image_title': 'Course Resources', 'image_description': 'Platforms used:\n-

In [12]:
out = json.dumps(message.content[0].input['steps'], indent=4)
def printout(jsonin):
    json_data = json.loads(jsonin)

    for dic in json_data:
        print(f"{dic['image_title']}: \n{dic['image_description']}\n")
printout(out)

Course Information Summary: 
CS 122A/EECS 116 Introduction to Data Management - Winter 2025
Instructor: Prof. Faisal Nawab
Lectures: TuTh 11AM-12:20PM at HSLH 100A
Office Hours: Tuesdays 12:30-1:30PM, DBH 2088

Course Components and Grading: 
- Homework Assignments (40%)
- Two Midterm Exams (20% each)
- Project (15%)
- Participation (5%)
Late Policy: Up to 2 homework assignments can be submitted up to 24 hours late using late passes

Learning Objectives: 
1. Design E-R diagrams
2. Build DB relations using relational data model
3. Apply database design principles
4. Use relational algebra
5. Create and use SQL
6. Understand indexing and physical database design

Course Resources: 
Platforms used:
- Canvas (primary resource)
- Gradescope (submissions)
- Google Drive (assignments)
- EdDiscussion (communication)
Textbook: Database Management Systems by Ramakrishnan & Gehrke (Optional but encouraged)

Academic Policies: 
- Zero-tolerance policy for cheating
- Individual work required on ass

In [13]:

message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=8192,
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "The following is a PDF syllabus document converted using aryn docparse. Please ingest the information of the course, and only return likely or defined prerequisites to ensure the student is prepared for the content of the class."
    f"{formatted_content}\n\n"
                }
            ],
        }
    ],
        tools=tools,
        tool_choice={"type": "tool", "name": "build_description_result"}

)
print(message)


Message(id='msg_01DQ4DMtJHByLMNDWJQViDy8', content=[ToolUseBlock(id='toolu_019zyX6uXuSJgV95YVG6Fkr7', input={'steps': [{'image_title': 'Prerequisites for CS 122A/EECS 116', 'image_description': 'The explicit prerequisite for this course is ICS 33 (with grade C or higher) OR EECS 114. Based on the course content which includes database design, SQL, and relational algebra, students should have a solid foundation in programming concepts and data structures which these prerequisites provide. The course feeds into more advanced database courses like CS122B, CS122C/CS222, and CS122D, but these are not prerequisites.'}]}, name='build_description_result', type='tool_use')], model='claude-3-5-sonnet-20241022', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=3939, output_tokens=158))


In [14]:

out = json.dumps(message.content[0].input['steps'], indent=4)


def printout(jsonin):
    json_data = json.loads(jsonin)

    for dic in json_data:
        print(f"{dic['image_title']}: \n{dic['image_description']}\n")


printout(out)

Prerequisites for CS 122A/EECS 116: 
The explicit prerequisite for this course is ICS 33 (with grade C or higher) OR EECS 114. Based on the course content which includes database design, SQL, and relational algebra, students should have a solid foundation in programming concepts and data structures which these prerequisites provide. The course feeds into more advanced database courses like CS122B, CS122C/CS222, and CS122D, but these are not prerequisites.
